In [7]:
import asyncio
import pandas as pd
from urllib.parse import quote
import feedparser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from googleapiclient.discovery import build
import os

load_dotenv()

# ✅ LLM 설정
llm = ChatOpenAI(model="gpt-4", temperature=0.1)

# ✅ 프롬프트
video_prompt = PromptTemplate.from_template(
    "너는 '덕순이'라는 캐릭터야.\n"
    "덕순이는 시니어 분들께 친절하게 정보를 전달하는 역할을 해.\n\n"
    "다음은 '{topic}'에 대한 유튜브 영상 검색 결과야.\n"
    "이 내용을 간단하고 친절한 말투로 요약해줘.\n\n"
    "👇 예시 형식 👇\n"
    "임영웅 관련 영상을 모아봤어요. 무대에서 감동적인 장면도 있었고, 팬들 사이에서 화제가 된 영상도 있어요.\n\n"
    "[LINK]\n"
    "1. https://youtube.com/link1\n"
    "2. https://youtube.com/link2\n"
    "3. https://youtube.com/link3\n\n"
    "이제 아래 내용을 요약해줘:\n"
    "{results}\n\n"
    "반드시 위의 형식을 보여줘!"
)

video_chain = video_prompt | llm

def youtube_search(query: str, max_results: int = 3) -> str:
    api_key = os.getenv("YOUTUBE_API_KEY")

    youtube = build("youtube", "v3", developerKey=api_key)

    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=max_results
    )

    response = request.execute()

    results = []

    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        title = item["snippet"]["title"]
        url = f"https://www.youtube.com/watch?v={video_id}"
        results.append(f"{title} - {url}")

    return "\n".join(results)

# ✅ GT 생성 함수
async def build_video_sample(question: str):
    try:
        results = youtube_search(question)
        if not results:
            return None

        input_data = {
            "topic": question,
            "results": "\n".join(results)  # LLM에 줄바꿈 포함해 전달
        }
        response = await video_chain.ainvoke(input_data)
        ground_truth = response.content.strip()

        return {
            "question": question,
            "video_links": results,  # 리스트 그대로!
            "ground_truth": ground_truth
        }

    except Exception as e:
        print(f"❌ [{question}] 처리 중 오류: {e}")
        return None

# ✅ 전체 실행 함수
async def generate_video_dataset(sample_questions: list[str]):
    print("🚀 유튜브 영상 요약 Ground Truth 생성 중...\n")
    tasks = [build_video_sample(q) for q in sample_questions]
    samples = await asyncio.gather(*tasks)
    samples = [s for s in samples if s]

    df = pd.DataFrame(samples)
    df.to_excel("video_ground_truth_dataset.xlsx", index=False)
    print("📁 저장 완료: video_ground_truth_dataset.xlsx")

# ✅ 질문 리스트 (예시 50개 생성)
sample_questions = []
idol_names = ["임영웅", "이찬원", "정동원", "영탁", "송가인"]

keywords = [
    "유튜브 영상", "최신 방송 영상 보여줘", "최근 콘서트 보여줘", "인터뷰 영상 찾아줘", 
    "감동 영상 추천해줘", "라이브 무대 보고싶어", "팬미팅 영상 찾아줘", "하이라이트 영상 보여줘", 
    "노래 모음 영상 알려줘", "최근 출연 방송 알려줘"
]

for name in idol_names:
    for kw in keywords:
        if len(sample_questions) < 50:
            sample_questions.append(f"{name} {kw}")



In [8]:
await generate_video_dataset(sample_questions)

🚀 유튜브 영상 요약 Ground Truth 생성 중...

📁 저장 완료: video_ground_truth_dataset.xlsx
